In [1]:
"""

190 page 기준 8분 정도
휴일 때 대략 100~200 page
평일 때 대략 400~500 page 

한 페이지에 15 기사

"""

'\n\n190 page 기준 8분 정도\n휴일 때 대략 100~200 page\n평일 때 대략 400~500 page \n\n한 페이지에 15 기사\n\n'

In [30]:
press_using_list = ["경향신문", "국민일보", "뉴스1", "뉴시스", "동아일보", "문화일보", "서울신문", "세계일보", "연합뉴스",  "조선일보", "중앙일보", "한겨레", "한국일보"]

In [73]:
from datetime import date
import pandas as pd
from tqdm import notebook
import requests
from bs4 import BeautifulSoup
import re

start_date = date(2022, 1, 1)
end_date = date(2022, 1, 15)


In [74]:
def create_soup(url):

    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
    return soup

In [184]:
def get_articles(year,month,day, category, press_using_list = None):
    
    no_get_list = ["@","무단전재", "재배포금지","저작권자 ⓒ 서울신문사","무단복제 및 전재","무단 전재 및 재배포","제보는 카톡", "☞", "무단 전재-재배포", "▶연합뉴스 앱 지금 바로 다운받기~"]
    replace_list = ["\xa0", "\u2008","\n             ","\n            ","\n          "]

    
    date_data = pd.DataFrame(columns=['arcid', 'press', 'title', 'content', 'journalist', 'date', 'link', 'category'])
    
    year = str(year)
    month = str(month).zfill(2)
    day = str(day).zfill(2)
    date = year+month+day
    
    
    # 마지막 페이지 번호 추출
    url = f"https://news.daum.net/breakingnews/{category}?page=999&regDate={date}"
    soup = create_soup(url)
    last_page = soup.find("em",attrs ={"class": "num_page"})
    last_page_num = int(re.sub(r'[^0-9]', '', last_page.text))

    
    # 본문 외 내용 제외를 위한 키워드 리스트
    # no_get_list = ["[카카오톡]", "[전화]", "[메일]","[온라인 제보]", "재배포 금지", "※ '당신의 제보가 뉴스가 됩니다'", "무단전재", "@"]
    
    
    for i in notebook.tqdm(range(1,last_page_num+1)):
        url = f"https://news.daum.net/breakingnews/society?page={i}&regDate={date}"
        soup = create_soup(url)
        
    
        # 뉴스 리스트가 포함된 속성 추출
        news_list_part = soup.find('ul', attrs={'class':'list_news2 list_allnews'})
        
        
        
        # COLUME : press
        press_list_ = []
        press = news_list_part.find_all('span', attrs={'class':'info_news'})
        for _span in press:
            press_list_.append(_span.text.split()[0])
 
        
        # 뉴스 링크 리스트 추출
        # COLUME : link
        link_list_ = []
        news_link_list = news_list_part.find_all('strong', attrs={'class':'tit_thumb'})
        for news_link in news_link_list:
            link_list_.append(news_link.find("a")["href"])
        
        
        
        
        # 안쓰는 언론사에 대해 모든 리스트에서 제거
        press_list = []
        link_list = []
        for idx in range(len(press_list_)):
            if press_list_[idx] in press_using_list:
                press_list.append(press_list_[idx])
                link_list.append(link_list_[idx])
                
                
        # COLUME : articleid
        articleid_list = []
        for link in link_list:
            articleid_list.append(link.split("/")[-1])
                
            
        
        

        # 링크마다 요청
        # COLUME : title, content ,journalist
        title_list = []
        journalist_list = []
        content_list = []

        for idx, link in enumerate(link_list):
            url = link
            article_soup = create_soup(url)
            article_soup = article_soup.find("article")
            
            ### title
            title = article_soup.find("h3", attrs={'class':"tit_view"})
            title_list.append(title.text)
            
            ### journalist
            journalist = article_soup.find("span", attrs={'class':"txt_info"})
            journalist_list.append(journalist.text)
            
            ### content
            content = article_soup.find("div", attrs={'class':"article_view"})
            
            # 이미지 제거 
            if content.select_one("p.link_figure") != None:
                content.select_one("p.link_figure").decompose()
                
            
            news_contents = content.find_all(True, attrs={'dmcf-ptype':'general'})
            news_content= ""
            
            # p text 확인 후 
            for _p in news_contents:
                skip_p = False
                
                tmp = _p.text
                
                # 주로 \과 관련된 문자를 " "로 변경
                for replace_word in replace_list:
                    tmp = tmp.replace(replace_word," ")
                    
                # tmp = tmp.replace(journalist_list[idx],"")
                
                # 예외가 포함된 문장을 건너뛰기
                for no in no_get_list :
                    if no in tmp :
                        skip_p = True
                        break;
                        
                if skip_p :
                    continue

                        
                news_content += tmp

                
            content_list.append(news_content)
            

        
        # 현페이지 데이터 프레임 생성
        data =  pd.DataFrame(columns=['arcid', 'press', 'title', 'content', 'journalist', 'date', 'link', 'category','drop'])
        data['arcid'] = articleid_list
        data['press'] = press_list
        data['title'] = title_list
        data['content'] = content_list
        data['journalist'] = journalist_list
        data['link'] = link_list
        
        # 저널리스트 입력 제거
        if len(journalist_list) > 0 :
            
            data["drop"] = data['journalist'].str.contains("입력")
            idx = data[data['drop'] == True].index
            data.drop(idx , inplace=True)

        data.drop(columns = "drop", inplace=True)
        
        
        date_data = pd.concat([date_data, data])
    
    
    
    date_data['date'] = date
    date_data['category'] = category

    date_data.reset_index(drop=True, inplace=True)
    
    return date_data

In [185]:
# [카카오톡] YTN을 검색해 채널 추가 [전화] 02-398-8585 [메일] social@ytn.co.kr [온라인 제보] www.ytn.co.kr

In [187]:
from dateutil.rrule import rrule, DAILY

df = pd.DataFrame()

for date in notebook.tqdm(list(rrule(DAILY, dtstart=start_date, until=end_date))):
    df = pd.concat([df, get_articles(date.year, date.month, date.day,"society",press_using_list)])
df.reset_index(drop=True, inplace=True)


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [165]:
df = get_articles(2022, 1, 2,"society",press_using_list)

  0%|          | 0/10 [00:00<?, ?it/s]

In [175]:
# """
# HTTPSConnectionPool(host='v.daum.net', port=443): Max retries exceeded with url: /v/20220101121016727 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020C9A0B23A0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))
# """


In [188]:
df["press"].value_counts()

뉴시스     143
뉴스1     109
연합뉴스    105
서울신문     61
조선일보     54
세계일보     49
경향신문     49
중앙일보     37
국민일보     24
한국일보     19
한겨레       9
문화일보      5
동아일보      4
Name: press, dtype: int64

In [ ]:
 ["경향신문", "국민일보", "뉴스1", "뉴시스", "동아일보", "문화일보", "서울신문", "세계일보", "연합뉴스",  "조선일보", "중앙일보", "한겨레", "한국일보"]